In [1]:
import pandas as pd
import numpy as np
import re
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import pickle

In [43]:
pays = pd.read_csv('/Volumes/Seagate/Galvanize/nj_payments_all_years_consl.csv', \
                   usecols=['company','amount','npi','year','payment_id'])

In [44]:
pays.head(2)

,payment_id,npi,company,amount,year
0,235088.0,1174629083,FOREST PHARMACEUTICALS,14.83,2013
1,334868.0,1386735850,FOREST PHARMACEUTICALS,72.90,2013


In [59]:
pay_by_comp = pays.groupby(['company']).agg({"amount": np.sum, "npi": pd.Series.nunique,'payment_id':'count'}).reset_index()
top10 = pay_by_comp.sort_values(['amount','npi','payment_id'],ascending=False).iloc[:10]

In [60]:
top10
 """),
    go.Bar(
        x=top10['company'],
        y=top10['npi'],
        name='Number Doctors Paid'
    ),
    go.Bar(
        x=top10['company'],
        y=top10['payment_id'],
        name='Number Payments'
    ),"""

,company,amount,npi,payment_id
82,ASTRAZENECA PHARMACEUTICALS,4037326.89,5358,63142
408,JANSSEN PHARMACEUTICALS,3570202.64,4683,45290
5,ABBVIE,3304520.11,2918,27077
857,ZIMMER BIOMET HOLDINGS,2829123.15,1015,3342
775,TEVA PHARMACEUTICALS USA,2165207.78,2034,18525
286,ER SQUIBB SONS,1960920.85,3277,20091
623,PFIZER,1945655.35,4571,34922
138,BOEHRINGER INGELHEIM PHARMACEUTICALS,1784204.80,3084,30656
760,TAKEDA PHARMACEUTICALS AMERICA,1713637.86,3233,24982
608,OTSUKA AMERICA PHARMACEUTICAL,1701228.73,1910,12052


In [66]:
data = [go.Bar(
        x=top10['company'],
        y=top10['amount'],
        name='Total Amount Paid',
        marker=dict(color=['FireBrick','Navy','Green','DarkMagenta',
                           'DarkOrange','Indigo','LightSkyBlue','DarkSlateGrey',
                           'MediumVioletRed','Peru']),)]

layout = go.Layout(
    title='Highest Paying Companies'
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.plot(fig,filename='top10companies')

In [11]:
meds = pd.read_csv('/Volumes/Seagate/Galvanize/Drug_Products_in_the_Medicaid_Drug_Rebate_Program.csv',
                  usecols=['NDC','Labeler Name','FDA Product Name','Year'])

In [12]:
meds.head(2)

,Year,Labeler Name,NDC,FDA Product Name
0,2014,ELI LILLY AND COMPANY,2120001,AMYVID ...
1,2014,ELI LILLY AND COMPANY,2140701,QUINIDINE GLUCONATE ...


In [23]:
#Removing excess spacing & any symbols
meds['FDA Product Name'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in meds['FDA Product Name']]
meds['FDA Product Name'] = meds['FDA Product Name'].str.strip()
meds['Labeler Name'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in meds['Labeler Name']]
meds['Labeler Name'] = meds['Labeler Name'].str.strip()

In [25]:
meds.head(2)

,Year,Labeler Name,NDC,FDA Product Name
0,2014,ELI LILLY AND COMPANY,2120001,AMYVID
1,2014,ELI LILLY AND COMPANY,2140701,QUINIDINE GLUCONATE


In [51]:
medsgrp = meds.groupby(['FDA Product Name','Labeler Name'])['NDC'].nunique().to_frame().reset_index()

In [52]:
print(len(medsgrp))
medsgrp.head()

45533


,FDA Product Name,Labeler Name,NDC
0,0000000000000000000000000000000000000ONIVYDE 4...,MERRIMACK PHARMACEUTICALS INC,1
1,0075 KCL IN 5 DEXTROSE AND 045 NACL INJECTION USP,B BRAUN MEDICAL INC,1
2,015 KCL IN 09 NACL INJECTION USP,B BRAUN MEDICAL INC,1
3,015 KCL IN 5 DEXTROSE AND 020 NACL INJECTION USP,B BRAUN MEDICAL INC,2
4,015 KCL IN 5 DEXTROSE AND 033 NACL INJECTION USP,B BRAUN MEDICAL INC,1


In [36]:
#Only getting brand name drugs
scripts = scripts[scripts['drug_name']!=scripts['generic_name']]

In [37]:
scriptsgrp = scripts.groupby('drug_name')['generic_name'].nunique().to_frame().reset_index()

In [49]:
print(len(scriptsgrp))
scriptsgrp.head()

1448


,drug_name,generic_name
0,ABILIFY,1
1,ABILIFY DISCMELT,1
2,ABILIFY MAINTENA,1
3,ABRAXANE,1
4,ABSTRAL,1


In [53]:
common = scriptsgrp.merge(medsgrp,left_on='drug_name',right_on='FDA Product Name')

In [55]:
scripts[(~scripts.drug_name.isin(common.drug_name))]

,drug_name,generic_name
0,VIAGRA,SILDENAFIL
4,ANDROGEL,TESTOSTERONE
7,VESICARE,SOLIFENACIN SUCCINATE
12,FLOMAX,TAMSULOSIN
15,RAPAFLO,SILODOSIN
19,AVODART,DUTASTERIDE
20,TOVIAZ,FESOTERODINE FUMARATE
53,CELEBREX,CELECOXIB
55,LYRICA,PREGABALIN
58,LIDODERM,LIDOCAINE VISCOUS


In [56]:
medsgrp['VIAGRA']

,FDA Product Name,Labeler Name,NDC


In [76]:
medsgrp[medsgrp['FDA Product Name'].str.contains('VIAGRA')].groupby('Labeler Name')['NDC'].count()

,FDA Product Name,Labeler Name,NDC
43965,VIAGRA SILDENAFIL CITRATE,PFIZER INC,5
43966,VIAGRA SILDENAFIL CITRATE 100MG TAB,PFIZER INC,2
43967,VIAGRA SILDENAFIL CITRATE 50MG TAB,PFIZER INC,2


In [77]:
scriptsgrp.to_csv('/Volumes/Seagate/Galvanize/brand_drugs_scripts.csv',index=False)

In [78]:
medsgrp.to_csv('/Volumes/Seagate/Galvanize/medicare_drugs.csv',index=False)

## <font color='blue'>Linking companies to their drugs</font>

In [2]:
import pandas as pd
import numpy as np

In [68]:
scriptsgrp= pd.read_csv('/Volumes/Seagate/Galvanize/brand_drugs_scripts.csv')

,drug_name,generic_name
0,ABILIFY,1
1,ABILIFY DISCMELT,1


In [4]:
medsgrp= pd.read_csv('/Volumes/Seagate/Galvanize/medicare_drugs.csv')

In [9]:
medsgrp[medsgrp['FDA Product Name'].str.contains('VIAGRA')].groupby('Labeler Name')['NDC'].count()

Labeler Name
PFIZER INC    3
Name: NDC, dtype: int64

Make a dict that has:

    keys = COMPANY
    values = ALL DRUGS
    
if the drug is in the k, v for that company:

    New Col [company] = Company

groupby company, iterate through company.unique, take company & index by it, for each company iterate through the drugs & add it to the values of that company

Map based on iff the drug is 

In [5]:
'VIAGRA'.str.contains('VIAGRA')

AttributeError: 'str' object has no attribute 'str'

In [5]:
viag = ['VIAGRA SILDENAFIL CITRATE','VIAGRA SILDENAFIL CITRATE 100MG TAB','VIAGRA SILDENAFIL CITRATE 50MG TAB']

In [7]:
viag

['VIAGRA SILDENAFIL CITRATE',
 'VIAGRA SILDENAFIL CITRATE 100MG TAB',
 'VIAGRA SILDENAFIL CITRATE 50MG TAB']

In [ ]:
lambda x: map(lambda w : v + w, y), x) 

for every drug:
    medsgrp[medsgrp['FDA Product Name'].str.contains(DRUG)].groupby('Labeler Name')['NDC'].count().to_frame().reset_index()

In [6]:
medsgrp[medsgrp['Labeler Name'].str.contains('VIDARA')]

,FDA Product Name,Labeler Name,NDC
601,ACTIMMUNE INTERFERON GAMMA1B,VIDARA THERAPEUTICS INC,2


In [35]:
x = 
        medsgrp[medsgrp['FDA Product Name'].str.contains('VIAGRA')].groupby('Labeler Name')['NDC'].count().to_frame(). \
        reset_index().sort_values('NDC',ascending=False).reset_index().drop('index',axis=1)['Labeler Name'][0]

In [19]:
d = {}
def company_drug(x):
    if x['drug_name'] not in d:
        try:
            df = medsgrp[medsgrp['FDA Product Name'].str.contains \
                                        (x['drug_name'])].groupby('Labeler Name')['NDC'].count()
            df = df.to_frame().reset_index().sort_values('NDC',ascending=False).reset_index().drop('index',axis=1)
            d[x['drug_name']] = df['Labeler Name'][0]
        except IndexError:
            df[x['drug_name']] = np.nan

In [21]:
scriptsgrp.apply(company_drug,axis=1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
1418    None
1419    None
1420    None
1421    None
1422    None
1423    None
1424    None
1425    None
1426    None
1427    None
1428    None
1429    None
1430    None
1431    None
1432    None
1433    None
1434    None
1435    None
1436    None
1437    None
1438    None
1439    None
1440    None
1441    None
1442    None
1443    None
1444    None
1445    None
1446    None
1447    None
Length: 1448, dtype: object

In [25]:
scriptsgrp["company"] = scriptsgrp["drug_name"].map(d)

In [45]:
medsgrp[medsgrp['FDA Product Name'].str.contains('ABSTRAL')].groupby('Labeler Name')['NDC'].count().to_frame(). \
        reset_index().sort_values('NDC',ascending=False).reset_index().drop('index',axis=1)['Labeler Name']

0        GALENA BIOPHARMA INC
1              PROSTRAKAN INC
2        SENTYNL THERAPEUTICS
3    SENTYNL THERAPEUTICS INC
Name: Labeler Name, dtype: object

In [18]:
medsgrp['Labeler Name'][0]

'MERRIMACK PHARMACEUTICALS INC'

In [34]:
script_and_company = scriptsgrp.copy()

In [35]:
script_company_dict = d

In [2]:
import pickle

In [38]:
script_and_company.fillna(value='UNKNOWN',inplace=True)

In [41]:
script_and_company.to_csv('/Volumes/Seagate/Galvanize/script_and_company.csv',index=False)

In [42]:
pickle.dump(script_company_dict, open('script_company_dict.pkl', 'wb'))

## <font color='teal'>Graphing work, mainly for graphing the amount a company made off a drug</font>

In [76]:
scripts = pd.read_csv('/volumes/Seagate/Galvanize/nj_scripts_all_years.csv',usecols=['drug_name','generic_name','amount_brand','total_drug_cost'])

In [95]:
comp_scripts = scripts[scripts['amount_brand']!=0]

In [89]:
comp_scripts.head()

,amount_brand,drug_name,generic_name,total_drug_cost
0,48,VIAGRA,SILDENAFIL,8957.33
4,29,ANDROGEL,TESTOSTERONE,17234.63
7,37,VESICARE,SOLIFENACIN SUCCINATE,13580.78
11,26,CIALIS,TADALAFIL,3348.48
12,23,FLOMAX,TAMSULOSIN,5060.75


In [96]:
comp_scripts["company"] = comp_scripts["drug_name"].map(d)

/Users/kunal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [97]:
comp_scripts.dropna(axis=0,how='any')

,amount_brand,drug_name,generic_name,total_drug_cost,company
0,48,VIAGRA,SILDENAFIL,8957.33,PFIZER INC
4,29,ANDROGEL,TESTOSTERONE,17234.63,ABBVIE INC
7,37,VESICARE,SOLIFENACIN SUCCINATE,13580.78,ASTELLAS
11,26,CIALIS,TADALAFIL,3348.48,ELI LILLY AND COMPANY
12,23,FLOMAX,TAMSULOSIN,5060.75,BOEHRINGER INGELHEIM PHARMACEUTICALS
15,16,RAPAFLO,SILODOSIN,4242.73,WATSON PHARMA INC
19,40,AVODART,DUTASTERIDE,6418.34,GLAXOSMITHKLINE
20,19,TOVIAZ,FESOTERODINE FUMARATE,7328.35,PFIZER INC
25,13,VIGAMOX,MOXIFLOXACIN,1320.04,ALCON LABORATORIES INC
26,41,LUMIGAN,BIMATOPROST,5446.44,ALLERGAN INC


In [98]:
scripts_by_comp = comp_scripts.groupby('company').agg({"total_drug_cost": np.sum})
top10 = scripts_by_comp.sort_values(['total_drug_cost'],ascending=False).round(decimals=2)
top10['total_drug_cost'] = top10.apply(lambda x: "{:,}".format(x['total_drug_cost']), axis=1)
top10['total_drug_cost'] = '$'+top10['total_drug_cost']
top10 = top10.reset_index().iloc[:10]

In [99]:
top10

,company,total_drug_cost
0,ASTRAZENECA LP,"$757,060,442.05"
1,GLAXOSMITHKLINE,"$369,410,690.79"
2,GILEAD SCIENCES INC,"$361,088,046.56"
3,MERCK CO INC,"$313,035,079.07"
4,BOEHRINGER INGELHEIM PHARMACEUTICALS,"$307,813,664.83"
5,NOVARTIS,"$290,281,177.46"
6,PFIZER INC,"$206,429,205.17"
7,AVENTIS PHARMACEUTICALS,"$203,123,038.16"
8,JANSSEN PHARMACEUTICALS INC,"$198,843,316.34"
9,FOREST LABORATORIES INC,"$186,255,882.94"


# **<font color='Red'>Trying to link the unknowns</font>**

In [3]:
script_company_dict = pickle.load(open('script_company_dict.pkl', 'rb'))

In [65]:
scriptcomp_orig = pd.read_csv('/Volumes/Seagate/Galvanize/script_and_company.csv')

In [36]:
scriptcomp.head(2)

,drug_name,generic_name,company
0,ABILIFY,1,OTSUKA AMERICA
1,ABILIFY DISCMELT,1,OTSUKA AMERICA


In [37]:
#Only getting hte unkonwn ones
scriptcomp = scriptcomp[scriptcomp['company']=='UNKNOWN']

In [19]:
comps = pd.read_csv('/Volumes/Seagate/Galvanize/ndcxls/product.csv', usecols=[1,3,12])

In [23]:
print(len(comps))
comps.head()

121214


,PRODUCTNDC,PROPRIETARYNAME,LABELERNAME
0,0002-1200,Amyvid,Eli Lilly and Company
1,0002-1407,Quinidine Gluconate,Eli Lilly and Company
2,0002-1433,Trulicity,Eli Lilly and Company
3,0002-1434,Trulicity,Eli Lilly and Company
4,0002-1445,TALTZ,Eli Lilly and Company


In [31]:
#Removing excess spacing & any symbols
comps['PROPRIETARYNAME'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in comps['PROPRIETARYNAME']]
comps['PROPRIETARYNAME'] = comps['PROPRIETARYNAME'].str.strip()
comps['LABELERNAME'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in comps['LABELERNAME']]
comps['LABELERNAME'] = comps['LABELERNAME'].str.strip()

In [32]:
medsgrp = comps.groupby(['PROPRIETARYNAME','LABELERNAME'])['PRODUCTNDC'].nunique().to_frame().reset_index()

In [33]:
print(len(medsgrp))
medsgrp.head()

69032


,PROPRIETARYNAME,LABELERNAME,PRODUCTNDC
0,01 MOISTURE FOUNDATION SPF 15,THE BODY SHOP WAKE FOREST,1
1,02 MOISTURE FOUNDATION SPF 15,THE BODY SHOP WAKE FOREST,1
2,03 MOISTURE FOUNDATION SPF 15,THE BODY SHOP WAKE FOREST,1
3,04 MOISTURE FOUNDATION SPF 15,THE BODY SHOP WAKE FOREST,1
4,05 MOISTURE FOUNDATION SPF 15,THE BODY SHOP WAKE FOREST,1


In [45]:
d = {}
def company_drug(x):
    if x['drug_name'] not in d:
        try:
            df = medsgrp[medsgrp['PROPRIETARYNAME'].str.contains \
                                        (x['drug_name'])].groupby('LABELERNAME')['PRODUCTNDC'].count()
            df = df.to_frame().reset_index().sort_values('PRODUCTNDC',ascending=False).reset_index().drop('index',axis=1)
            d[x['drug_name']] = df['LABELERNAME'][0]
        except IndexError:
            d[x['drug_name']] = np.nan

In [46]:
scriptcomp.apply(company_drug,axis=1)

19      None
27      None
34      None
46      None
53      None
72      None
75      None
85      None
96      None
109     None
110     None
122     None
123     None
126     None
136     None
137     None
143     None
144     None
160     None
161     None
163     None
167     None
169     None
206     None
211     None
212     None
213     None
225     None
226     None
239     None
        ... 
1281    None
1282    None
1283    None
1284    None
1288    None
1289    None
1294    None
1295    None
1299    None
1303    None
1307    None
1308    None
1310    None
1325    None
1326    None
1327    None
1334    None
1335    None
1352    None
1354    None
1355    None
1356    None
1357    None
1360    None
1361    None
1372    None
1385    None
1426    None
1432    None
1433    None
Length: 235, dtype: object

In [70]:
#Compiling the dictionary
comp_dict = {**script_company_dict, **d}

In [73]:
d['ADACEL TDAP'] = 'SANOFI PASTEUR INC'

In [83]:
#Add newly linked company names to the df
scriptcomp["company"] = scriptcomp["drug_name"].map(d)

In [115]:
scriptcomp.head()

,drug_name,generic_name,company
19,ADACEL TDAP,1,SANOFI PASTEUR INC
27,ADVOCATE PEN NEEDLES,1,NaN
34,AK POLY BAC,1,NaN
46,ALPRAZOLAM ODT,1,NaN
53,ALZ NAC,1,NaN


In [84]:
left = scriptcomp[pd.isnull(scriptcomp['company'])]
len(left),len(scriptcomp_orig)

(190, 1448)

In [85]:
pickle.dump(comp_dict, open('script_company_dict.pkl', 'wb'))

## Linking to the OG scripts dataframe

In [89]:
scripts = pd.read_csv('/volumes/Seagate/Galvanize/nj_scripts_all_years.csv')

In [90]:
scripts.head(2)

,npi,ln,fn,city,state,specialty_description,year,recieved_payments,total_claim_count,amount_brand,drug_name,generic_name,brand_drug?,total_day_supply,total_drug_cost
0,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,48,48,VIAGRA,SILDENAFIL,True,1481,8957.33
1,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,16,0,CLOTRIMAZOLE BETAMETHASONE,CLOTRIMAZOLE BETAMETHASONE,False,367,806.59


In [91]:
#Add newly linked company names to the df
scripts["drug_company"] = scripts["drug_name"].map(comp_dict)

In [106]:
#Writing unknown where I couln't link the company, and generic for the company
dic = {}
def unknown_and_generic(x):
    if (x['drug_name'] == x['generic_name']):
            dic[x['drug_name']] = 'GENERIC'
    elif (x['drug_name'] != x['generic_name']) and (x['drug_name'] not in dic):
        dic[x['drug_name']] = 'UNKNOWN'

In [107]:
scriptsgrp = scripts[pd.isnull(scripts['drug_company'])]
scriptsgrp = scriptsgrp.groupby(['drug_name','generic_name'])['npi'].count().to_frame().reset_index()

In [108]:
scriptsgrp.apply(unknown_and_generic,axis=1)

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
       ... 
924    None
925    None
926    None
927    None
928    None
929    None
930    None
931    None
932    None
933    None
934    None
935    None
936    None
937    None
938    None
939    None
940    None
941    None
942    None
943    None
944    None
945    None
946    None
947    None
948    None
949    None
950    None
951    None
952    None
953    None
Length: 954, dtype: object

In [111]:
#Add newly linked company names to the df
drug_comp = {**comp_dict, **dic}
scripts["drug_company"] = scripts["drug_name"].map(drug_comp)

In [112]:
scripts

,npi,ln,fn,city,state,specialty_description,year,recieved_payments,total_claim_count,amount_brand,drug_name,generic_name,brand_drug?,total_day_supply,total_drug_cost,drug_company
0,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,48,48,VIAGRA,SILDENAFIL,True,1481,8957.33,PFIZER INC
1,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,16,0,CLOTRIMAZOLE BETAMETHASONE,CLOTRIMAZOLE BETAMETHASONE,False,367,806.59,GENERIC
2,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,27,0,IMIPRAMINE,IMIPRAMINE,False,1380,519.91,GENERIC
3,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,34,0,NITROFURANTOIN,NITROFURANTOIN,False,1480,1502.16,GENERIC
4,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,29,29,ANDROGEL,TESTOSTERONE,True,1236,17234.63,ABBVIE INC
5,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,28,0,ACETAMINOPHEN CODEINE,ACETAMINOPHEN CODEINE,False,106,119.43,GENERIC
6,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,35,0,OXYBUTYNIN CHLORIDE,OXYBUTYNIN CHLORIDE,False,2070,667.09,GENERIC
7,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,37,37,VESICARE,SOLIFENACIN SUCCINATE,True,2145,13580.78,ASTELLAS
8,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,15,0,AMOXICILLIN,AMOXICILLIN,False,124,70.21,GENERIC
9,1528036670,LANTERI,VINCENT,MAYWOOD,NJ,Urology,2013,True,111,0,CIPROFLOXACIN,CIPROFLOXACIN,False,889,645.97,GENERIC


In [129]:
scripts.to_csv('/volumes/Seagate/Galvanize/nj_scripts_all_years.csv',index=False)

In [114]:
pickle.dump(drug_comp, open('script_company_dict.pkl', 'wb'))

In [119]:
def clean_drug_name(df,wrong_name, right_name):
    """
    Input:
        df: df, dataframe to be used
        wrong_name: Str, wrong name that is listed in df
        right_name: Str, right name to be changed to
    Output:
        None
    """
    #Getting all the locations where it says the wrong name
    for col in ['drug_name']:
        l = list(df[(df['drug_name']==wrong_name)&(df['generic_name']==right_name)].index.values)
        for i in l:
            df.at[i,'drug_name'] = right_name

In [120]:
clean_drug_name(scripts,'ALPRAZOLAM ODT','ALPRAZOLAM INTENSOL')

In [126]:
l = list(scripts[(scripts['drug_name']=='ALPRAZOLAM INTENSOL')&(scripts['generic_name']=='ALPRAZOLAM INTENSOL')].index.values)

In [134]:
scripts['drug_company'] = scripts['drug_company'].map(lambda x: ' '.join((str(x).replace('INCORPORATED','').replace('CORPORATION','')\
                                    .replace('INC','').replace('CORP','').replace('LLC','').replace('LP','')).split()))

In [159]:
pd.options.display.max_rows = 60

In [139]:
def clean_company_name(df,wrong_name, right_name):
    """
    Input:
        df: df, dataframe to be used
        wrong_name: Str, wrong name that is listed in df
        right_name: Str, right name to be changed to
    Output:
        None
    """
    #Getting all the locations where it says the wrong name
    l = list(df[df['drug_company']==wrong_name].index.values)
    #Replacing all the "ZOLL SERVICES AKA ZOLL LIFECOR" with "ZOLL LIFECOR"
    for i in l:
        df.at[i,'drug_company'] = right_name

In [140]:
clean_company_name(scripts,'WYETH LABORATORIES','PFIZER')
clean_company_name(scripts,'WYETH PHARMACEUTICALS A SUBSIDIARY OF PFIZER','PFIZER')

In [160]:
pd.DataFrame(scripts['drug_company'].unique()).sort_values(by=0)

,0
204,3M ESPE DENTAL PRODUCTS
16,ABBOTT LABORATORIES
2,ABBVIE
206,ABRAXIS BIOSCIENCE
169,ACELLA PHARMACEUTICALS
109,ACORDA THERAPEUTICS
38,ACTAVIS
138,ACTELION PHARMACEUTICALS US
170,ACTIENT PHARMACEUTICALS
247,ACTON PHARMACEUTICALS


In [157]:
clean_company_name(scripts,'ACTAVIS KADIAN','ACTAVIS')
clean_company_name(scripts,'ACTAVIS PHARMA','ACTAVIS')
clean_company_name(scripts,'ARBOR PHARMACEUTICALS IRELAND LIMITED','ARBOR PHARMACEUTICALS')
clean_company_name(scripts,'ASTELLAS PHARMA US','ASTELLAS')
clean_company_name(scripts,'BRISTOLMYERS SQUIBB AND GILEAD SCIENCE','GILEAD SCIENCES')
clean_company_name(scripts,'BRISTOLMYERS SQUIBB COMPANY','BRISTOLMYERS SQUIBB')
clean_company_name(scripts,'BRISTOLMYERS SQUIBBSANOFI PARTNERSHIP','BRISTOLMYERS SQUIBB')
clean_company_name(scripts,'BRISTOLMYERS SQUIBB PHARMA CO','BRISTOLMYERS SQUIBB')
clean_company_name(scripts,'COVIS PHARMA SARL','COVIS PHARMACEUTICALS')
clean_company_name(scripts,'CSL BEHRING GMBH','CSL BEHRING')
clean_company_name(scripts,'GLAXOSMITHKLINE BIOLOGICALS SA','GLAXOSMITHKLINE')
clean_company_name(scripts,'GSK CONSUMER HEALTHCARE','GSK CONSUMER HEALTH')
clean_company_name(scripts,'IMPAX SPECIALTY PHARMA','IMPAX LABORATORIES')
clean_company_name(scripts,'JANSSEN BIOTECH','JANSSEN PHARMACEUTICALS')
clean_company_name(scripts,'JANSSEN PRODUCTS','JANSSEN PHARMACEUTICALS')
clean_company_name(scripts,'JAZZ PHARMACEUTICALS COMMERCIAL','JAZZ PHARMACEUTICALS')
clean_company_name(scripts,'KREMERS URBAN','KREMERS URBAN PHARMACEUTICALS')
clean_company_name(scripts,'MALLKRODT BRAND PHARMACEUTICALS','MALLKRODT PHARMACEUTICALS')
clean_company_name(scripts,'MALLKRODT','MALLKRODT PHARMACEUTICALS')
clean_company_name(scripts,'MERCK SHARP DOHME','MERCK')
clean_company_name(scripts,'MERCKSCHERINGPLOUGH JV','MERCK')
clean_company_name(scripts,'MYLAN INSTITUTIONAL','MYLAN PHARMACEUTICALS')
clean_company_name(scripts,'MYLAN SPECIALTY','MYLAN PHARMACEUTICALS')
clean_company_name(scripts,'PAR PHARMACEUTICAL','PAR PHARMACEUTICALS')
clean_company_name(scripts,'SCHERING HEALTHCARE PRODUCTS','SCHERING')
clean_company_name(scripts,'TEVA GLOBAL RESPIRATORY RESEARCH','TEVA PHARMACEUTICALS')
clean_company_name(scripts,'TEVA NEUROSCIENCE','TEVA PHARMACEUTICALS')
clean_company_name(scripts,'TEVA PARENTERAL MEDICINES','TEVA PHARMACEUTICALS')
clean_company_name(scripts,'TEVA PHARMACEUTICALS USA','TEVA PHARMACEUTICALS')
clean_company_name(scripts,'TEVA RESPIRATORY','TEVA PHARMACEUTICALS')
clean_company_name(scripts,'TEVA WOMENS HEALTH','TEVA PHARMACEUTICALS')
clean_company_name(scripts,'UCB','UCB PHARMA')
clean_company_name(scripts,'UCB MANUFACTURING','UCB PHARMA')

In [150]:
x = ayments_nj_full.company.unique()

In [151]:
sorted(x)

['180 MEDICAL',
 '3M COMPANY',
 'AASTROM BIOSCIENCES',
 'ABB CONCISE OPTICAL GROUP',
 'ABBOTT LABORATORIES',
 'ABBVIE',
 'ABIOMED',
 'ACCESS CLOSURE',
 'ACCLARENT',
 'ACCURAY',
 'ACE SURGICAL SUPPLY CO',
 'ACELL',
 'ACIST MEDICAL SYSTEMS',
 'ACORDA THERAPEUTICS',
 'ACTAVIS PHARMA',
 'ACTELION CLINICAL RESEARCH',
 'ACTELION PHARMACEUTICALS LTD',
 'ACTELION PHARMACEUTICALS US',
 'ACUMED',
 'ADEC',
 'ADVANCED MEDICAL PARTNERS',
 'ADVANCED ORTHOPAEDIC SOLUTIONS',
 'ADVANCED RESPIRATORY',
 'ADVANDX',
 'AEGERION PHARMACEUTICALS',
 'AEROCRINE',
 'AESCULAP',
 'AESCULAP BIOLOGICS',
 'AESCULAP IMPLANT SYSTEMS',
 'AHATEC SPINE',
 'AKORN',
 'AKRIMAX PHARMACEUTICALS',
 'ALCON LABORATORIES',
 'ALCON RESEARCH LTD',
 'ALEXION PHARMACEUTICALS',
 'ALGETA US',
 'ALIGN TECHNOLOGY',
 'ALIMERA SCIENCES',
 'ALKABELLO',
 'ALKERMES',
 'ALLERGAN',
 'ALLIQUA BIOMEDICAL',
 'ALMATICA PHARMA',
 'ALTATEC GMBH',
 'ALTUS PARTNERS',
 'AMAG PHARMACEUTICALS',
 'AMARIN PHARMA',
 'AMD GROUP',
 'AMD LASERS',
 'AMENDIA',
 'A

In [162]:
len(np.intersect1d(sorted(scripts['drug_company'].unique()),sorted(x)))

113

In [163]:
scripts.to_csv('/volumes/Seagate/Galvanize/nj_scripts_all_years.csv',index=False)

In [2]:
doc_info = pd.read_csv('/Volumes/Seagate/Galvanize/nj_doc_info.csv',dtype={'Zip Code':object,'NPI':object})
doc_info.fillna(value='-',inplace=True)
paid = pd.read_csv('/Volumes/Seagate/Galvanize/nj_payments_all_years_consl.csv',
                            dtype={'zip':object,'npi':object,'company_id':object}, \
                  usecols=[1,2,3,10,11,12,13,26])

In [3]:
paid.head(2)

,npi,fn,ln,company,amount,form,nature,year
0,1174629083,FATIMA,ASGHAR,FOREST PHARMACEUTICALS,14.83,In-kind items and services,Food and Beverage,2013
1,1386735850,RAGHUVEER,ANNAM,FOREST PHARMACEUTICALS,72.90,In-kind items and services,Food and Beverage,2013


In [9]:
len(doc_info)

26429

In [5]:
notpaid = list(set(doc_info.NPI)-set(paid.npi))

In [10]:
doc_info_paid = doc_info[~doc_info['NPI'].isin(notpaid)]

In [11]:
doc_info_paid.to_csv('/Volumes/Seagate/Galvanize/nj_doc_info_paid.csv',index=False)

In [52]:
script = pd.read_csv('/Volumes/Seagate/Galvanize/nj_scripts_all_years.csv',dtype={'zip':object})

In [20]:
script.head(2)

,amount_brand,brand_drug?,city,drug_company,drug_name,fn,generic_name,ln,npi,recieved_payments,specialty_description,state,total_claim_count,total_day_supply,total_drug_cost,year
0,48,True,MAYWOOD,PFIZER,VIAGRA,VINCENT,SILDENAFIL,LANTERI,1528036670,True,Urology,NJ,48,1481,8957.33,2013
1,0,False,MAYWOOD,NaN,CLOTRIMAZOLE BETAMETHASONE,VINCENT,CLOTRIMAZOLE BETAMETHASONE,LANTERI,1528036670,True,Urology,NJ,16,367,806.59,2013


In [24]:
doc_info

,amount_brand,brand_drug?,city,drug_company,drug_name,fn,generic_name,ln,npi,recieved_payments,specialty_description,state,total_claim_count,total_day_supply,total_drug_cost,year
0,48,True,MAYWOOD,PFIZER,VIAGRA,VINCENT,SILDENAFIL,LANTERI,1528036670,True,Urology,NJ,48,1481,8957.33,2013
1,0,False,MAYWOOD,NaN,CLOTRIMAZOLE BETAMETHASONE,VINCENT,CLOTRIMAZOLE BETAMETHASONE,LANTERI,1528036670,True,Urology,NJ,16,367,806.59,2013


In [32]:
l = list(script[script['brand_drug?'] == False].index.values)

In [33]:
for i in list(script[script['brand_drug?'] == False].index.values):
    script.at[i,'drug_company'] = 'GENERIC'

In [ ]:
for i in list(script[(script['brand_drug?'] == True) & ].index.values)sc:
    script.at[i,'drug_company'] = 'GENERIC'

In [34]:
scscript[(script['brand_drug?'] == True) & (script['brand_drug?'] not in dic)]

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [38]:
x = script[~script['drug_name'].isin(dic)]

In [39]:
x[x['brand_drug?']==True]

,amount_brand,brand_drug?,city,drug_company,drug_name,fn,generic_name,ln,npi,recieved_payments,specialty_description,state,total_claim_count,total_day_supply,total_drug_cost,year
248904,12,True,VOORHEES,NaN,TRANXENE T TAB,EDWARD,CLORAZEPATEOTASSIUM,SKOBAC,1285607671,False,Internal Medicine,NJ,12,360,4409.84,2013
256689,14,True,EAST BRUNSWICK,NaN,BLEPHAMIDE S O P,KENNETH,SULFACETM NA PREDNISOL AC,YANG,1114985595,True,Ophthalmology,NJ,14,157,1188.78,2013
269138,12,True,WOODSTOWN,NaN,TRI PREVIFEM,JAMES,NORGESTIMATE ETHINYL ESTRADIOL,HUBBS,1922078906,False,Family Practice,NJ,12,342,248.40,2013
295512,12,True,EATONTOWN,NaN,NEPHRO VITE RX,ALAN,VIT B CMPLX 3 FA VIT C BIOTIN,HARATZ,1336155688,True,Nephrology,NJ,12,360,41.90,2013
296850,11,True,HAMMONTON,NaN,DILATRATE SR,ANTHONY,ISOSORBIDE DINITRATE,SALVO,1841236080,True,Family Practice,NJ,11,330,885.90,2013
300606,11,True,VINELAND,NaN,ERY TAB,ASHOK,ERYTHROMYCIN,PILLY,1801997523,True,Cardiology,NJ,11,182,910.67,2013
307829,12,True,TRENTON,NaN,MONO LINYAH,SADIA,NORGESTIMATE ETHINYL ESTRADIOL,QAZI,1467491563,True,Internal Medicine,NJ,12,336,247.40,2013
320041,18,True,LITTLE FALLS,NaN,BIFERA RX,JEFFREY,IRON PS IRON HEM POLY FA B12,FARNESE,1730269994,False,Internal Medicine,NJ,18,540,1257.63,2013
322073,14,True,PLEASANTVILLE,NaN,MONOJECT INSULIN SAFETY SYRNG,CHRISTINE,SYRINGE W NDL DISP INSULIN,ABLETT,1134452832,False,Nurse Practitioner,NJ,14,350,573.86,2013
325465,13,True,TOMS RIVER,NaN,LOW OGESTREL,CORINNE,NORGESTREL ETHINYL ESTRADIOL,MALMBERG,1023054574,False,Nurse Practitioner,NJ,13,364,297.70,2013


In [40]:
for i in list(x[x['brand_drug?']==True].index.values):
    script.at[i,'drug_company'] = 'UNKNOWN'

In [41]:
script.to_csv('/Volumes/Seagate/Galvanize/nj_scripts_all_years.csv',index=False)

In [49]:
doc_info = pd.read_csv('/Volumes/Seagate/Galvanize/nj_doc_info_paid.csv',dtype={'Zip Code':object,'NPI':object})
info_df = doc_info[doc_info['NPI']=='1477533925']

In [50]:
info_df

,NPI,First Name,Last Name,Type,Gender,Address,City,State,Zip Code
2591,1477533925,LOWELL,KABNICK,M.D.,M,95 MADISON AVE,MORRISTOWN,NJ,07960


In [48]:
list(x['First Name'].values)[0]

'LOWELL'

In [51]:
list(info_df['Last Name'].values)[0]

'KABNICK'

In [53]:
script.head()

,amount_brand,brand_drug?,city,drug_company,drug_name,fn,generic_name,ln,npi,recieved_payments,specialty_description,state,total_claim_count,total_day_supply,total_drug_cost,year
0,48,True,MAYWOOD,PFIZER,VIAGRA,VINCENT,SILDENAFIL,LANTERI,1528036670,True,Urology,NJ,48,1481,8957.33,2013
1,0,False,MAYWOOD,GENERIC,CLOTRIMAZOLE BETAMETHASONE,VINCENT,CLOTRIMAZOLE BETAMETHASONE,LANTERI,1528036670,True,Urology,NJ,16,367,806.59,2013
2,0,False,MAYWOOD,GENERIC,IMIPRAMINE,VINCENT,IMIPRAMINE,LANTERI,1528036670,True,Urology,NJ,27,1380,519.91,2013
3,0,False,MAYWOOD,GENERIC,NITROFURANTOIN,VINCENT,NITROFURANTOIN,LANTERI,1528036670,True,Urology,NJ,34,1480,1502.16,2013
4,29,True,MAYWOOD,ABBVIE,ANDROGEL,VINCENT,TESTOSTERONE,LANTERI,1528036670,True,Urology,NJ,29,1236,17234.63,2013


In [57]:
list(script[script['npi']==1528036670]['specialty_description'].values)[0]

'Urology'

In [2]:
scripts = pd.read_csv('/Volumes/Seagate/Galvanize/nj_scripts_all_years.csv',dtype={'zip':object,'npi':object})
scripts.head(2)

,amount_brand,brand_drug?,drug_name,generic_name,npi,city,fn,ln,state,recieved_payments,specialty_description,total_claim_count,total_day_supply,total_drug_cost,year
0,48,True,VIAGRA,SILDENAFIL,1528036670,MAYWOOD,VINCENT,LANTERI,NJ,True,Urology,48,1481,8957.33,2013
1,0,False,CLOTRIMAZOLE BETAMETHASONE,CLOTRIMAZOLE BETAMETHASONE,1528036670,MAYWOOD,VINCENT,LANTERI,NJ,True,Urology,16,367,806.59,2013


## For some reason I don't have the comp column, going to relink

In [4]:
dic = pickle.load(open('/Volumes/Seagate/Galvanize/pickles/script_company_dict.pkl', 'rb'))

In [9]:
type(dic)

dict

In [31]:
def unknown_and_generic(x):
    if (x['drug_name'] == x['generic_name']):
            dic[x['drug_name']] = 'GENERIC'
    elif (x['drug_name'] != x['generic_name']) and (x['drug_name'] not in dic):
            dic[x['drug_name']] = 'UNKNOWN'

In [32]:
s = scripts.groupby(['drug_name','generic_name'])['total_claim_count'].sum()
s = s.to_frame().reset_index()

In [33]:
s = s[~s['drug_name'].isin(dic.keys())]

In [34]:
s.apply(unknown_and_generic,axis=1)

0       None
1       None
3       None
4       None
10      None
11      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
23      None
35      None
36      None
38      None
40      None
44      None
50      None
57      None
60      None
61      None
62      None
65      None
66      None
70      None
72      None
75      None
78      None
        ... 
2319    None
2328    None
2331    None
2338    None
2339    None
2340    None
2348    None
2349    None
2361    None
2370    None
2373    None
2374    None
2376    None
2377    None
2381    None
2382    None
2386    None
2392    None
2393    None
2398    None
2420    None
2422    None
2423    None
2448    None
2450    None
2459    None
2461    None
2463    None
2471    None
2477    None
Length: 1047, dtype: object

In [36]:
'CLOTRIMAZOLE BETAMETHASONE' in dic

True

In [37]:
scripts["drug_company"] = scripts["drug_name"].map(dic)

In [41]:
x = scripts[scripts['drug_company'].isnull()]

In [42]:
x.head()

,amount_brand,brand_drug?,drug_name,generic_name,npi,city,fn,ln,state,recieved_payments,specialty_description,total_claim_count,total_day_supply,total_drug_cost,year,drug_company


In [ ]:
scriptsgrp2 = scripts[pd.isnull(scripts['drug_company'])]
scriptsgrp2 = scriptsgrp2.groupby(['drug_name','generic_name'])['npi'].count().to_frame().reset_index()

In [43]:
scripts.to_csv('/Volumes/Seagate/Galvanize/nj_scripts_all_years.csv',index=False)

In [44]:
scripts.head()

,amount_brand,brand_drug?,drug_name,generic_name,npi,city,fn,ln,state,recieved_payments,specialty_description,total_claim_count,total_day_supply,total_drug_cost,year,drug_company
0,48,True,VIAGRA,SILDENAFIL,1528036670,MAYWOOD,VINCENT,LANTERI,NJ,True,Urology,48,1481,8957.33,2013,PFIZER INC
1,0,False,CLOTRIMAZOLE BETAMETHASONE,CLOTRIMAZOLE BETAMETHASONE,1528036670,MAYWOOD,VINCENT,LANTERI,NJ,True,Urology,16,367,806.59,2013,GENERIC
2,0,False,IMIPRAMINE,IMIPRAMINE,1528036670,MAYWOOD,VINCENT,LANTERI,NJ,True,Urology,27,1380,519.91,2013,GENERIC
3,0,False,NITROFURANTOIN,NITROFURANTOIN,1528036670,MAYWOOD,VINCENT,LANTERI,NJ,True,Urology,34,1480,1502.16,2013,GENERIC
4,29,True,ANDROGEL,TESTOSTERONE,1528036670,MAYWOOD,VINCENT,LANTERI,NJ,True,Urology,29,1236,17234.63,2013,ABBVIE INC


In [45]:
s3 = pd.read_csv('/Volumes/Seagate/Galvanize/nj_scripts_all_years.csv',dtype={'zip':object,'npi':object})
s3.head(2)

,amount_brand,brand_drug?,drug_name,generic_name,npi,city,fn,ln,state,recieved_payments,specialty_description,total_claim_count,total_day_supply,total_drug_cost,year,drug_company
0,48,True,VIAGRA,SILDENAFIL,1528036670,MAYWOOD,VINCENT,LANTERI,NJ,True,Urology,48,1481,8957.33,2013,PFIZER INC
1,0,False,CLOTRIMAZOLE BETAMETHASONE,CLOTRIMAZOLE BETAMETHASONE,1528036670,MAYWOOD,VINCENT,LANTERI,NJ,True,Urology,16,367,806.59,2013,GENERIC


In [47]:
s3[s3['npi']=='1386639722']

,amount_brand,brand_drug?,drug_name,generic_name,npi,city,fn,ln,state,recieved_payments,specialty_description,total_claim_count,total_day_supply,total_drug_cost,year,drug_company
622271,0,False,AMOXICILLIN,AMOXICILLIN,1386639722,PARAMUS,MARK,HARTZBAND,NJ,True,Orthopedic Surgery,17,108,89.67,2013,GENERIC
622272,24,True,LYRICA,PREGABALIN,1386639722,PARAMUS,MARK,HARTZBAND,NJ,True,Orthopedic Surgery,24,550,4043.07,2013,PFIZER INC
622273,0,False,HYDROCODONE ACETAMINOPHEN,HYDROCODONE ACETAMINOPHEN,1386639722,PARAMUS,MARK,HARTZBAND,NJ,True,Orthopedic Surgery,18,192,324.64,2013,GENERIC
622274,83,True,CELEBREX,CELECOXIB,1386639722,PARAMUS,MARK,HARTZBAND,NJ,True,Orthopedic Surgery,83,3185,30676.98,2013,PFIZER INC
622275,0,False,ENOXAPARIN SODIUM,ENOXAPARIN SODIUM,1386639722,PARAMUS,MARK,HARTZBAND,NJ,True,Orthopedic Surgery,21,166,4374.44,2013,GENERIC
622276,0,False,TRAMADOL,TRAMADOL,1386639722,PARAMUS,MARK,HARTZBAND,NJ,True,Orthopedic Surgery,26,277,178.12,2013,GENERIC
622277,0,False,OXYCODONE ACETAMINOPHEN,OXYCODONE ACETAMINOPHEN,1386639722,PARAMUS,MARK,HARTZBAND,NJ,True,Orthopedic Surgery,41,301,352.86,2013,GENERIC
1265838,0,False,HYDROCODONE ACETAMINOPHEN,HYDROCODONE ACETAMINOPHEN,1386639722,PARAMUS,MARK,HARTZBAND,NJ,True,Orthopaedic Surgery,18,215,879.60,2014,GENERIC
1265839,0,False,AMOXICILLIN,AMOXICILLIN,1386639722,PARAMUS,MARK,HARTZBAND,NJ,True,Orthopaedic Surgery,27,143,99.89,2014,GENERIC
1265840,29,True,LYRICA,PREGABALIN,1386639722,PARAMUS,MARK,HARTZBAND,NJ,True,Orthopaedic Surgery,29,780,7339.52,2014,PFIZER INC


In [48]:
paid = pd.read_csv('/Volumes/Seagate/Galvanize/nj_payments_all_years_consl.csv',
                            dtype={'zip':object,'npi':object,'company_id':object}, \
                  usecols=[1,2,3,10,11,12,13,26])

In [102]:
df = paid.copy()
npi = '1386639722'
pay_doc = df[df['npi']==str(npi)].groupby(['npi','fn','ln','year']).agg({'amount':np.sum, 'form':'count'}).reset_index()

In [103]:
pay_doc['avg'] = (pay_doc['amount']/pay_doc['form']).round(2)

In [94]:
pay_doc

,npi,fn,ln,year,amount,form,avg
0,1386639722,MARK,HARTZBAND,2013,212641.10,56,3797.16
1,1386639722,MARK,HARTZBAND,2014,462742.01,93,4975.72
2,1386639722,MARK,HARTZBAND,2015,501312.80,37,13548.99
3,1386639722,MARK,HARTZBAND,2016,750325.56,83,9040.07


In [84]:
pay_doc['avg'] = pay_doc.apply(lambda x: "{:,}".format(x['avg']), axis=1)
pay_doc['amount'] = pay_doc.apply(lambda x: "{:,}".format(x['amount']), axis=1)
pay_doc

,npi,fn,ln,year,amount,form,Average Payment Amount
0,1386639722,MARK,HARTZBAND,2013,"212,641.1",56,"3,797.16"
1,1386639722,MARK,HARTZBAND,2014,"462,742.01",93,"4,975.72"
2,1386639722,MARK,HARTZBAND,2015,"501,312.8",37,"13,548.99"
3,1386639722,MARK,HARTZBAND,2016,"750,325.56",83,"9,040.07"


In [90]:
pay_doc.columns

Index(['npi', 'fn', 'ln', 'year', 'amount', 'form', 'Average Payment Amount'], dtype='object')

In [104]:
pay_doc['amount'] = pay_doc['amount'].astype(str)

In [105]:
pay_doc

,npi,fn,ln,year,amount,form,avg
0,1386639722,MARK,HARTZBAND,2013,212641.10000000003,56,3797.16
1,1386639722,MARK,HARTZBAND,2014,462742.0099999999,93,4975.72
2,1386639722,MARK,HARTZBAND,2015,501312.8000000001,37,13548.99
3,1386639722,MARK,HARTZBAND,2016,750325.56,83,9040.07


In [108]:
len(df.form.unique()),len(df.nature.unique())

(5, 14)

In [109]:
df.form.unique()

array(['In-kind items and services', 'Cash or cash equivalent',
       'Stock, stock option, or any other ownership interest',
       'Dividend, profit or other return on investment', 'Stock'],
      dtype=object)

In [110]:
df.nature.unique()

array(['Food and Beverage', 'Education', 'Travel and Lodging',
       'Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program',
       'Charitable Contribution', 'Consulting Fee', 'Honoraria',
       'Entertainment', 'Royalty or License',
       'Compensation for serving as faculty or as a speaker for a non-accredited and noncertified continuing education program',
       'Grant', 'Gift',
       'Current or prospective ownership or investment interest',
       'Compensation for serving as faculty or as a speaker for an accredited or certified continuing education program'],
      dtype=object)

In [111]:
nature_yr = df[df['npi']==str(npi)]

In [118]:
x = nature_yr.groupby(['year','nature']).agg({'amount':np.sum}).reset_index()
y = x[x.year == 2013]

In [119]:
y

,year,nature,amount
0,2013,Compensation for serving as faculty or as a sp...,69500.00
1,2013,Consulting Fee,45250.00
2,2013,Education,6.28
3,2013,Food and Beverage,1210.24
4,2013,Royalty or License,93517.37
5,2013,Travel and Lodging,3157.21


In [121]:
x.nature.unique()

array(['Compensation for serving as faculty or as a speaker for a non-accredited and noncertified continuing education program',
       'Consulting Fee', 'Education', 'Food and Beverage',
       'Royalty or License', 'Travel and Lodging',
       'Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program'],
      dtype=object)

In [122]:
nature_yr = df[df['npi']==str(npi)].groupby(['year','nature']).agg({'amount':np.sum}).reset_index()


In [124]:
df.head(2)

,npi,fn,ln,company,amount,form,nature,year
0,1174629083,FATIMA,ASGHAR,FOREST PHARMACEUTICALS,14.83,In-kind items and services,Food and Beverage,2013
1,1386735850,RAGHUVEER,ANNAM,FOREST PHARMACEUTICALS,72.90,In-kind items and services,Food and Beverage,2013


In [125]:
pay_doc = df[df['npi']==str(npi)].groupby(['npi','fn','ln','year']).agg({'amount':np.sum, 'form':'count'}).reset_index()
pay_doc['avg'] = (pay_doc['amount']/pay_doc['form'])
pay_doc = pay_doc.round(2)
pay_doc['avg'] = pay_doc['avg']

In [126]:
pay_doc

,npi,fn,ln,year,amount,form,avg
0,1386639722,MARK,HARTZBAND,2013,212641.10,56,3797.16
1,1386639722,MARK,HARTZBAND,2014,462742.01,93,4975.72
2,1386639722,MARK,HARTZBAND,2015,501312.80,37,13548.99
3,1386639722,MARK,HARTZBAND,2016,750325.56,83,9040.07


In [127]:
nature_yr = df[df['npi']==str(npi)].groupby(['year','nature']).agg({'amount':np.sum}).reset_index()
nature_yr

,year,nature,amount
0,2013,Compensation for serving as faculty or as a sp...,69500.00
1,2013,Consulting Fee,45250.00
2,2013,Education,6.28
3,2013,Food and Beverage,1210.24
4,2013,Royalty or License,93517.37
5,2013,Travel and Lodging,3157.21
6,2014,Compensation for services other than consultin...,91633.79
7,2014,Compensation for serving as faculty or as a sp...,33000.00
8,2014,Consulting Fee,42000.00
9,2014,Education,24.40


In [128]:
nall = df[df['npi']==str(npi)].groupby(['nature']).agg({'amount':np.sum}).reset_index()

In [129]:
nall

,nature,amount
0,Compensation for services other than consultin...,91633.79
1,Compensation for serving as faculty or as a sp...,122500.00
2,Consulting Fee,217000.00
3,Education,30.68
4,Food and Beverage,7097.32
5,Royalty or License,1461226.47
6,Travel and Lodging,27533.21


In [148]:
pay_doc = df[df['npi']==str(npi)].groupby(['npi','fn','ln','year']).agg({'amount':np.sum, 'form':'count'}).reset_index()
pay_doc['year'] = pay_doc['year'].astype(str)
pay_doc = pay_doc.append(pay_doc.sum(numeric_only=True), ignore_index=True)
pay_doc.fillna({'year':'All Years'},inplace=True)
pay_doc['avg'] = (pay_doc['amount']/pay_doc['form'])
pay_doc=pay_doc.round(2)
pay_doc['amount'] = pay_doc['amount']
pay_doc

,npi,fn,ln,year,amount,form,avg
0,1386639722,MARK,HARTZBAND,2013,0 212641.10\n1 462742.01\n2 501312...,56.0,3797.16
1,1386639722,MARK,HARTZBAND,2014,0 212641.10\n1 462742.01\n2 501312...,93.0,4975.72
2,1386639722,MARK,HARTZBAND,2015,0 212641.10\n1 462742.01\n2 501312...,37.0,13548.99
3,1386639722,MARK,HARTZBAND,2016,0 212641.10\n1 462742.01\n2 501312...,83.0,9040.07
4,NaN,NaN,NaN,All Years,0 212641.10\n1 462742.01\n2 501312...,269.0,7163.65


In [139]:
type(pay_doc['avg'][0])

numpy.float64

In [142]:
def dec(x):
    if type(x) == np.float64:
        return('{:.2f}'.format(x))

In [149]:
df= pd.read_csv('/Volumes/Seagate/Galvanize/nj_scripts_all_years.csv',dtype={'npi':object})

In [170]:
grp = df[df['specialty_description']=='Ophthalmology'].groupby(['recieved_payments']).agg({'amount_brand':'sum','total_claim_count':'sum'}).reset_index()
spec_df = grp.copy()

In [171]:
spec_df['%_brand'] = (spec_df['amount_brand']/spec_df['total_claim_count'])

In [172]:
spec_df

,recieved_payments,amount_brand,total_claim_count,%_brand
0,False,195862,472853,0.414213
1,True,1196900,2388751,0.501057
